In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [10]:
data = pd.read_csv('NSTU_Students_data_transformed.csv').drop('Курс обучения', axis=1)

In [11]:
results_dict = {}
best_params_dict = {}

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 37 columns):
 #   Column                                                                                                 Non-Null Count  Dtype 
---  ------                                                                                                 --------------  ----- 
 0   Пол                                                                                                    426 non-null    bool  
 1   Сумма баллов ЕГЭ за 3 предмета                                                                         426 non-null    int64 
 2   Принимали ли вы участие в олимпиадах по профильным предметам? / Математика                             426 non-null    bool  
 3   Принимали ли вы участие в олимпиадах по профильным предметам? / Информатика и программирование         426 non-null    bool  
 4   Принимали ли вы участие в олимпиадах по профильным предметам? / Физика                                 4

In [13]:
X, y = data.drop('target', axis = 1), data['target']

In [14]:
categorical_features = X.columns[X.dtypes == 'object'].tolist()
binary_features = X.columns[(X.nunique() == 2) & (X.dtypes != 'object')].tolist()

In [16]:
binary_features

['Пол',
 'Принимали ли вы участие в олимпиадах по профильным предметам? / Математика',
 'Принимали ли вы участие в олимпиадах по профильным предметам? / Информатика и программирование',
 'Принимали ли вы участие в олимпиадах по профильным предметам? / Физика',
 'Какие увлечения у вас были, когда вы учились в школе? / Геймдев, VR, AR',
 'Какие увлечения у вас были, когда вы учились в школе? / Спорт',
 'Какие увлечения у вас были, когда вы учились в школе? / Программирование',
 'Какие увлечения у вас были, когда вы учились в школе? / Робототехника',
 'Какие увлечения у вас были, когда вы учились в школе? / 3D-моделирование',
 'Какие увлечения у вас были, когда вы учились в школе? / Решение математических задач',
 'Какие увлечения у вас были, когда вы учились в школе? / Творчество: музыка, рисование, танцы и т.п.',
 'Какие увлечения у вас были, когда вы учились в школе? / Физика',
 'Вы поступили в университет, потому что сами этого хотели?',
 'Вам важно получать хорошие отметки?',
 'QA-те

In [17]:
def evaluate_model(name, model, X, y):
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    accuracy = scores.mean()
    f1 = cross_val_score(model, X, y, cv=5, scoring='f1').mean()
    precision = cross_val_score(model, X, y, cv=5, scoring='precision').mean()
    recall = cross_val_score(model, X, y, cv=5, scoring='recall').mean()
    
    results_dict[name] = {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall
    }

# Random forest

In [19]:
rf_pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        [('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)])),
    ('classifier', RandomForestClassifier(random_state=42))
])

rf_params = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
}

rf_grid = GridSearchCV(rf_pipeline, rf_params, cv=5, scoring='accuracy', n_jobs=-1)
rf_grid.fit(X, y)

evaluate_model("Random Forest", rf_grid.best_estimator_, X, y)
best_params_dict['Random Forest'] = rf_grid.best_params_

ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "c:\Users\user\AppData\Local\Programs\

In [18]:
results_dict

{}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Загружаем данные (примерно), делим на признаки и целевую переменную
X = data.drop('target', axis=1)
y = data['target']

# Определим категориальные признаки
categorical_features = X.columns[X.dtypes == 'object'].tolist()
binary_features = X.columns[(X.nunique() == 2) & (X.dtypes != 'object')].tolist()

# Словари для хранения результатов
results_dict = {}
best_params_dict = {}

# Функция для сохранения метрик
def evaluate_model(name, model, X, y):
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    accuracy = scores.mean()
    f1 = cross_val_score(model, X, y, cv=5, scoring='f1').mean()
    precision = cross_val_score(model, X, y, cv=5, scoring='precision').mean()
    recall = cross_val_score(model, X, y, cv=5, scoring='recall').mean()

    # Сохраняем метрики в словарь
    results_dict[name] = {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall
    }

# 1. Random Forest
rf_pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        [('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)])),
    ('classifier', RandomForestClassifier(random_state=42))
])

rf_params = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
}

rf_grid = GridSearchCV(rf_pipeline, rf_params, cv=5, scoring='accuracy', n_jobs=-1)
rf_grid.fit(X, y)

# Сохраняем результаты и лучшие параметры
evaluate_model("Random Forest", rf_grid.best_estimator_, X, y)
best_params_dict['Random Forest'] = rf_grid.best_params_

# 2. Gradient Boosting
gb_pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        [('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)])),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

gb_params = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': [3, 5],
}

gb_grid = GridSearchCV(gb_pipeline, gb_params, cv=5, scoring='accuracy', n_jobs=-1)
gb_grid.fit(X, y)

# Сохраняем результаты и лучшие параметры
evaluate_model("Gradient Boosting", gb_grid.best_estimator_, X, y)
best_params_dict['Gradient Boosting'] = gb_grid.best_params_

# 3. CatBoost (без кодирования категорий)
cat_features_indices = [X.columns.get_loc(col) for col in categorical_features + binary_features]

cat_params = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 5],
}

cat_model = CatBoostClassifier(cat_features=cat_features_indices, random_state=42, verbose=0)
cat_grid = GridSearchCV(cat_model, cat_params, cv=5, scoring='accuracy', n_jobs=-1)
cat_grid.fit(X, y)

# Сохраняем результаты и лучшие параметры
evaluate_model("CatBoost", cat_grid.best_estimator_, X, y)
best_params_dict['CatBoost'] = cat_grid.best_params_

# Выводим результаты
print("Результаты метрик для каждой модели:")
for model_name, metrics in results_dict.items():
    print(f"{model_name}: {metrics}")

print("\nЛучшие параметры для каждой модели:")
for model_name, params in best_params_dict.items():
    print(f"{model_name}: {params}")
